In [1]:
import pandas as pd
import csv
from PIL import Image
import requests

# Assignment 1 - Store Inventory 

For this assignment you want to build an inventory, such as you might find at a store. There are two key objects that you'll need here (though you may do more if you want/need):

<b>Products:</b>
<ul>
<li> Each item should be a product object, containing: </li>
    <ul>
    <li> A unique id number </li>
    <li> A product name </li>
    <li> A category </li>
    <li> A subcategory </li>
    <li> The URL of an image</li>
    <li> The URL of the product </li>
    <li> A rating (0-5) </li>
    <li> The number of reviews that constituted the rating </li>
    <li> A regular price </li>
    <li> A sale price </li>
    </ul>
<li> Products should have several methods, not limited to, but including: </li>
    <ul>
    <li> A __str__ function that prints the product in a nice format, displaying: </li>
        <ul>
        <li> The product name </li>
        <li> The image </li>
        <li> The product rating </li>
        <li> The product price </li>
        </ul>
    <li> A method to set the discount price - that can accept either a price or a percentage discount as an argument and calculate a new discount price from the actual price </li>
    <li> A method to add a rating to a product </li>
    </ul>
</ul>

<b>Inventory:</b>
<ul>
<li> Each inventory object should keep track of all the products along with a stock level. </li>
<li> The inventory should include methods such as:</li>
    <ul>
    <li> A method to add a new product to the inventory </li>
    <li> A method to remove a product from the inventory </li>
    <li> A method to print all the products in a category in a nice format </li>
    <li> A method to use the + sign to add two inventories together </li>
    <li> A method to get the Top X highest rated products </li>
    <li> A method to change the stock levels for any item in that inventory </li>
    <li> An overload of the length method, to return the number of items in the inventory </li>
    </ul>
</ul>

## Marks and Key Points

There are marks allocated for the following:
<ul>
<li> <b>20%</b> - Error checking - use try/except blocks to catch errors. Specifically check the image loading (some URLs may not work) and the price cleanup (the data may be unclean). </li>
<li> <b>30%</b> - Use of classes and methods - you should have at least the classes constructed with relevant methods. </li>
<li> <b>30%</b> - Output - I will test your code with data that is different, but in the exact same format as the sample. Everything you create should be generic enough to handle any data in the same format. </li>
    <ul>
    <li> At the bottom of your code, please make a section that shows that it works. Load two data files into two inventory objects, display some contents from each, add rating, show the highest rated objects, change a stock level, and add the two inventories together. </li>
    <li> In here, please be clear in what you are showing. Use titled markup blocks to illustrate. You're basically showing, "here's it working". </li>
    </ul>
<li> <b>20%</b> - Code comments and formatting - you should have comments throughout your code explaining what you are doing. Use markup blocks to add a description. </li>
</ul>

### Tips and Hints

<b>Note:</b> I will update the README file in the original repository if I need to add any notes and clarifications. That's likely, as this is obviously open-ended. In general, if you need to make reasonable assumptions, please do so. 

<ul>
<li> The "Show Images" block below shows an example of loading an image from a URL. </li>
<li> Check the parts of error checking other than try/except, they may be useful, depending on what you choose. </li>
<li> You can use the CSV files in the repsoitory for testing, each should work. </li>
<li> If something is really unclear, please ask. </li>
</ul>

#### I was not able to get the inventory class working properly to read the files but I did my best to fill everything else out as correctly as I could.


## Classes

In [61]:
class myProduct():
    def __init__(self, product_id, name, category, sub_category, image, link, rating, no_of_ratings, discount_price, full_price):
        self.name = name
        self.category = category
        self.sub_category = sub_category
        self.image = image
        self.link = link
        self.rating = rating
        self.no_of_ratings = no_of_ratings
        self.discount_price = discount_price
        self.full_price = full_price
        self.product_id = product_id
    
    def add_rating(self, new_rating):
        #Takes the old rating and multiplies by the number of rating to get the sum of the ratings, after which a new rating can be added in and a new average rating can be calculated
        total = self.rating * self.no_of_ratings
        final_rating = (new_rating + total)/(self.no_of_ratings + 1)
        return final_rating
    
    def set_discount(self, discount):
        #Apply the discount to the full price, then change the discount price to this new value
        new_discount_price = self.full_price - ((discount/100) * self.full_price) 
        self.discount_price = new_discount_price
        return self.discount_price
    
    def show_image(self, filename, image, product_id):
        #Display an image based on the location of the product in the inventory, unless it's not in the right format
        df = pd.read_csv(filename)
        try: 
            image = df["image"][product_id-1]
            im = Image.open(requests.get(image, stream=True).raw)
            display(im)
        except:
            print("Error: Image is not valid.")
            
    def __str__(self):
        img = self.show_image(self.image)
        ret_str = f"{self.name}: {img} Product rating: {self.rating}, Product Price: {self.discount_price}"
        return ret_str
    

In [59]:
class myInventory():
    def __init___(self, stock_level):
        self.products = []
        self.stock_level = stock_level
    def read_file(self, filename):
        self.products = []
        try:
            with open(filename, "r",encoding='utf-8') as file:
                #Splitting the csv file by each comma after each line of code, ignoring commas within descriptions
                csv_file = csv.reader(file, delimiter=',', quotechar='"')
                #Ignoring the first line which is the column names
                header = next(csv_file)
                #Reading each line and applying each item in that line to each value of product
                for line in csv_file:
                    #Assigning a unique id number to each product
                    product_id = 1
                    name, category, sub_category, image, link, rating, no_of_ratings, discount_price, full_price = line
                    #To avoid errors while reading the file, replacing all symbols and commas with blank spaces
                    product = myProduct(product_id, name, category, sub_category, image, link, float(rating), float(no_of_ratings.replace(',', '')), float(discount_price.replace('₹', '').replace(',', '')), float(full_price.replace('₹', '').replace(',', '')))
                    self.products.append(product)
                    product_id += 1
        except FileNotFoundError:
            print(f"File '{filename}' not found.")
    
    def add_product(self, name, category, sub_category, image, link, rating, no_of_ratings, discount_price, full_price):
        #Adjust the inventory class with a new product with all the relevant information
        try:
            new_product = myProduct(name, category, sub_category, image, link, rating, no_of_ratings, discount_price, full_price)
            self.products.append(new_product)
            return self.products
        except:
            print("Error: Product information not viable or compatible.")
    def remove_product(self, product_id):
        #Remove a product from the list of products
        self.products.remove(product_id)
        return self.products

    def print_inv(self):
        #Return the list of inventory products
        try:
            return self.products()
        except:
            print("Error: Product inventory not viable or in proper format.")

    def __add__(self, filename):
        #Combine two inventories together
        try:
            new_inventory = myInventory()
            new_inventory.read_file(filename)
            self.products.append(new_inventory)
            return self.products
        except:
            print("Error: Inventory not viable or in correct format.")
    def print_top_rated(self, number):
        #Sort all the products based on rating, then select the number of products requested
        top_rated = []
        sorted_inventory = sorted(self.products, key=lambda x: x[self.rating])
        i = 0
        while i >= number:
            top_rated.append(sorted_inventory[i])
            i += 1
        return top_rated

    def adjust_stock(self, prod_id, stock_amount):
        #Increase the stock level of the specified product id
        try:
            for i in self.products:
                if self.product_id == prod_id:
                    self.stock_level += stock_amount
                else:
                    i += 1
            return self.products
        except:
            print("Error: Product ID not valid.")
    def __len__(self):
        #Overload the len function to return the number of items in the list
        length = len(self.products)
        return length

## Testing

This stuff should work...

You should also add more testing, that can be built into your final part of code. 

In [ ]:
df = pd.read_csv("Strength Training.csv")
print(df)

                                                   name     main_category  \
0              Protoner 16Kg PVC Combo Home Gym Package  sports & fitness   
1     Unisex Hexagon Rubber Vinyl Fixed Dumbbell - P...  sports & fitness   
2     LAFILLETTE 150 lbs Metal Hand Gripper Forearm ...  sports & fitness   
3     Aurion Genuine Leather Pro Weight Lifting Belt...  sports & fitness   
4     SPIRO PVC (Round/Hex) Dumbbells (1 Kg. X 2 = 2...  sports & fitness   
...                                                 ...               ...   
1099  Body Flow Fitness Olympic Clips, Clamps, Colla...  sports & fitness   
1100  GYM INSANE Gym Equipment Set for Home Gym Work...  sports & fitness   
1101  ROCKFIT PVC 1KG to 5KG Dumbbell Set for Home G...  sports & fitness   
1102  Urdhvamurti 3 in 1 Multi Usage 5 Detachable Ch...  sports & fitness   
1103  Arcado 5kg Dumbbells Set of 2 / Gym Dumbels Se...  sports & fitness   

           sub_category                                              image 

In [ ]:
df1 = pd.read_csv("Car Electronics.csv")
df.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,Protoner 16Kg PVC Combo Home Gym Package,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/71K-gcsnOY...,https://www.amazon.in/Protoner-16Kg-PVC-Combo-...,3.6,917,₹965.08,"₹4,999"
1,Unisex Hexagon Rubber Vinyl Fixed Dumbbell - P...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/71pcrSXprs...,https://www.amazon.in/Generic-Vinyl-Coated-Iro...,4.4,367,₹389,"₹6,450"
2,LAFILLETTE 150 lbs Metal Hand Gripper Forearm ...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/41oTg0XNbF...,https://www.amazon.in/LALA-LIFE-Gripper-Streng...,4.5,381,₹489,"₹1,199"
3,Aurion Genuine Leather Pro Weight Lifting Belt...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/813g2hRYuo...,https://www.amazon.in/Genuine-Comfortable-Adju...,4.4,446,₹799,₹899
4,SPIRO PVC (Round/Hex) Dumbbells (1 Kg. X 2 = 2...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/61VcW8otph...,https://www.amazon.in/Dumbbells-Equipment-Fitn...,3.5,52,₹82.31,₹200


In [ ]:
with open("Strength Training.csv") as f:
    for line in f:
        print(line)

name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price

Protoner 16Kg PVC Combo Home Gym Package,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/71K-gcsnOYL._AC_UL320_.jpg,https://www.amazon.in/Protoner-16Kg-PVC-Combo-Package/dp/B08HZ7MY7T/ref=sr_1_97?qid=1679218003&s=sports&sr=1-97,3.6,917,â‚¹965.08,"â‚¹4,999"

"Unisex Hexagon Rubber Vinyl Fixed Dumbbell - Pair Set Hand Weights for Strength Training - Weight Loss, Workout Bench, Gym...",sports & fitness,Strength Training,https://m.media-amazon.com/images/I/71pcrSXprsL._AC_UL320_.jpg,https://www.amazon.in/Generic-Vinyl-Coated-Iron-Dumbbells/dp/B07PM1ZMHK/ref=sr_1_98?qid=1679218003&s=sports&sr=1-98,4.4,367,â‚¹389,"â‚¹6,450"

LAFILLETTE 150 lbs Metal Hand Gripper Forearm Wrist Heavy Strength Exercise Hand Grip/Fitness Grip (Black),sports & fitness,Strength Training,https://m.media-amazon.com/images/I/41oTg0XNbFL._AC_UL320_.jpg,https://www.amazon.in/LALA-LIFE-Gripper-Strength-Ex

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 7816: character maps to <undefined>

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1104 entries, 0 to 1103
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            1104 non-null   object
 1   main_category   1104 non-null   object
 2   sub_category    1104 non-null   object
 3   image           1104 non-null   object
 4   link            1104 non-null   object
 5   ratings         897 non-null    object
 6   no_of_ratings   897 non-null    object
 7   discount_price  1046 non-null   object
 8   actual_price    1088 non-null   object
dtypes: object(9)
memory usage: 77.8+ KB


In [ ]:
# Create an inventory from the supplied file. 
inv = myInventory()
inv.read_file("Strength Training.csv")

In [ ]:
#inv.printNext()

In [ ]:
##inv.printTopRated()

Test 2

In [ ]:
inv2 = myInventory()
#inv2.read_file("Car Electronics.csv")
#inv2.printNext()

In [ ]:
#new = inv + inv2
#new.printNext()